In [1]:
import sys
assert sys.version_info >= (3, 5)

In [2]:
import sys
assert sys.version_info >= (3, 5)

In [3]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"


In [6]:
excel_file = 'train.xlsx'
data = pd.read_excel(excel_file)
(data.head())

,date,close
0,2018-11-13,1631.17
1,2018-11-12,1636.85
2,2018-11-09,1712.43
3,2018-11-08,1754.91
4,2018-11-07,1755.49


In [7]:
data = data.sort_values(by='date')
(data.head(12))

,date,close
1499,2012-11-30,252.0500
1498,2012-12-03,250.3291
1497,2012-12-04,252.4881
1496,2012-12-05,253.9600
1495,2012-12-06,253.3700
1494,2012-12-07,253.2681
1493,2012-12-10,247.7700
1492,2012-12-11,250.6900
1491,2012-12-12,251.7600
1490,2012-12-13,251.2500


In [8]:
data.drop(['date'], axis=1)

,close
1499,252.0500
1498,250.3291
1497,252.4881
1496,253.9600
1495,253.3700
...,...
4,1755.4900
3,1754.9100
2,1712.4300
1,1636.8500


In [9]:
training_set = data.iloc[:,1:2].values

In [10]:
training_set

array([[ 252.05  ],
       [ 250.3291],
       [ 252.4881],
       ...,
       [1712.43  ],
       [1636.85  ],
       [1631.17  ]])

In [11]:
from sklearn.preprocessing import MinMaxScaler 
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(training_set)

In [28]:
scaled_data

array([[0.00382873],
       [0.00286965],
       [0.00407289],
       ...,
       [0.81771469],
       [0.77559312],
       [0.77242759]])

In [25]:
X_train = []
y_train = []
for i in range(11, 1500):
    X_train.append(scaled_data[i-11:i,0])
    y_train.append(scaled_data[i,0])
X_train, y_train = np.array(X_train), np.array(y_train)

In [26]:
y_train

array([0.00483746, 0.00848227, 0.00713916, ..., 0.81771469, 0.77559312,
       0.77242759])

In [14]:
X_train.shape

(1489, 11)

In [15]:
y_train.shape
#Y = np.expand_dims(Y, axis=1)
#Y.shape

(1489,)

In [16]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [17]:
X_train.shape

(1489, 11, 1)

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.


In [19]:
model = Sequential()

In [20]:

model.add(LSTM(units=50, return_sequences= True, input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 11, 50)            10400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 11, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 11, 50)            20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 11, 50)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 11, 50)            20200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 11, 50)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)               

In [21]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [22]:
model.fit(X_train, y_train, epochs=20, batch_size=25)

Epoch 1/20
1489/1489 [==============================] - 12s 8ms/step - loss: 0.0209
Epoch 2/20
1489/1489 [==============================] - 5s 3ms/step - loss: 0.0028
Epoch 3/20
1489/1489 [==============================] - 4s 3ms/step - loss: 0.0019
Epoch 4/20
1489/1489 [==============================] - 4s 2ms/step - loss: 0.0019
Epoch 5/20
1489/1489 [==============================] - 4s 2ms/step - loss: 0.0017
Epoch 6/20
1489/1489 [==============================] - 4s 2ms/step - loss: 0.0017A: 1s - loss: 0 - ETA: 1s - ETA: 0s - l
Epoch 7/20
1489/1489 [==============================] - 4s 2ms/step - loss: 0.0015
Epoch 8/20
1489/1489 [==============================] - 4s 3ms/step - loss: 0.0014
Epoch 9/20
1489/1489 [==============================] - 4s 3ms/step - loss: 0.0015
Epoch 10/20
1489/1489 [==============================] - 4s 3ms/step - loss: 0.0015
Epoch 11/20
1489/1489 [==============================] - 4s 2ms/step - loss: 0.0019
Epoch 12/20
1489/1489 [======================

In [23]:
model.evaluate(X_train, y_train)

1489/1489 [==============================] - 2s 1ms/step


0.0004020309193926701

In [17]:
excel_file1 = 'test.xlsx'
test_data = pd.read_excel(excel_file1)
(test_data.head())


,date,close
0,2019-11-12,1780.10
1,2019-11-11,1771.65
2,2019-11-08,1785.88
3,2019-11-07,1788.20
4,2019-11-06,1795.77


In [18]:
data1 = test_data.sort_values(by='date')
(data1.head(12))

,date,close
249,2018-11-14,1599.01
248,2018-11-15,1619.44
247,2018-11-16,1593.41
246,2018-11-19,1512.29
245,2018-11-20,1495.46
244,2018-11-21,1516.73
243,2018-11-23,1502.06
242,2018-11-26,1581.33
241,2018-11-27,1581.42
240,2018-11-28,1677.75


In [19]:
data1 = data1.drop('date',axis = 1)

In [20]:
(data1.head())

,close
249,1599.01
248,1619.44
247,1593.41
246,1512.29
245,1495.46


In [21]:
test_set = data1.iloc[:,0:1].values

In [22]:
test_set


array([[1599.01],
       [1619.44],
       [1593.41],
       [1512.29],
       [1495.46],
       [1516.73],
       [1502.06],
       [1581.33],
       [1581.42],
       [1677.75],
       [1673.57],
       [1690.17],
       [1772.36],
       [1668.4 ],
       [1699.19],
       [1629.13],
       [1641.03],
       [1643.24],
       [1663.54],
       [1658.38],
       [1591.91],
       [1520.91],
       [1551.48],
       [1495.08],
       [1460.83],
       [1377.45],
       [1343.96],
       [1470.9 ],
       [1461.64],
       [1478.02],
       [1501.97],
       [1539.13],
       [1500.28],
       [1575.39],
       [1629.51],
       [1656.58],
       [1659.42],
       [1656.22],
       [1640.56],
       [1617.21],
       [1674.56],
       [1683.78],
       [1693.22],
       [1696.2 ],
       [1632.17],
       [1640.02],
       [1654.93],
       [1670.57],
       [1637.89],
       [1593.88],
       [1670.43],
       [1718.73],
       [1626.23],
       [1633.31],
       [1658.81],
       [16

In [23]:
test_scaled_data = scaler.fit_transform(test_set)

In [26]:
test_scaled_data

array([[0.3767189 ],
       [0.40689482],
       [0.36844748],
       [0.24863005],
       [0.22377147],
       [0.2551881 ],
       [0.23351993],
       [0.35060485],
       [0.35073778],
       [0.49302099],
       [0.48684696],
       [0.51136582],
       [0.63276369],
       [0.47921067],
       [0.52468871],
       [0.42120733],
       [0.4387841 ],
       [0.44204836],
       [0.47203226],
       [0.46441074],
       [0.36623192],
       [0.26136213],
       [0.30651522],
       [0.2232102 ],
       [0.1726216 ],
       [0.04946605],
       [0.        ],
       [0.18749538],
       [0.173818  ],
       [0.1980119 ],
       [0.233387  ],
       [0.28827378],
       [0.2308908 ],
       [0.34183123],
       [0.42176861],
       [0.46175206],
       [0.46594686],
       [0.46122033],
       [0.43808989],
       [0.40360102],
       [0.48830923],
       [0.50192754],
       [0.51587079],
       [0.52027237],
       [0.42569753],
       [0.43729229],
       [0.45931495],
       [0.482

In [27]:
X_test = []
y_test = []
for i in range(11, 250):
    X_test.append(test_scaled_data[i-11:i, 0])
    y_test.append(test_scaled_data[i, 0])
X_test, y_test = np.array(X_test), np.array(y_test)

In [28]:
X_test[0]

array([0.3767189 , 0.40689482, 0.36844748, 0.24863005, 0.22377147,
       0.2551881 , 0.23351993, 0.35060485, 0.35073778, 0.49302099,
       0.48684696])

In [33]:
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [34]:
X_test.shape

(239, 11, 1)

In [35]:
model.evaluate(X_test,y_test)

239/239 [==============================] - 0s 644us/step


0.011487262251774223